<a href="https://colab.research.google.com/github/axel-sirota/practical-nlp/blob/main/1-synonims/Practical_NLP_1_CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk

TRACE = False
embedding_dim = 50
epochs=100
batch_size = 500
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config) 
  K.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

Overwriting get_data.sh


In [4]:
!bash get_data.sh

In [5]:
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]
X = yelp_best_worst.text
y = yelp_best_worst.stars.map({1:0, 5:1})

In [6]:
corpus = [sentence for sentence in X.values if type(sentence) == str and len(TextBlob(sentence).words) > 3]


In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
tokenized_corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in tokenized_corpus)
vocab_size = len(tokenizer.word_index) + 1


In [8]:
print(f'First 5 corpus items are {tokenized_corpus[:5]}')
print(f'Length of corpus is {len(tokenized_corpus)}')



First 5 corpus items are [[12, 447, 202, 35, 41, 20, 12, 571, 11, 282, 2, 9, 8, 196, 1, 1549, 8, 201, 71, 123, 654, 319, 4500, 43, 2394, 58, 1408, 1478, 50, 483, 8, 196, 2, 50, 28, 572, 664, 20, 1, 3444, 458, 616, 450, 9, 388, 38, 1, 27, 4501, 53, 178, 664, 25, 1, 1631, 15, 46, 41, 1, 138, 85, 600, 4, 1632, 2, 46, 43, 2217, 2726, 9, 8, 1388, 2, 693, 1, 66, 74, 109, 23, 86, 178, 163, 17, 77, 356, 632, 45, 43, 1036, 2, 2395, 80, 130, 54, 15, 113, 9, 9, 8, 99, 170, 140, 20, 1, 122, 545, 196, 3, 23, 1, 475, 2218, 3230, 770, 1409, 2727, 2, 9, 8, 301, 2, 108, 9, 154, 16, 144, 859, 6, 43, 8190, 243, 16, 8, 99, 2, 9, 364, 123, 1, 179, 998, 9, 8, 1, 66, 812, 74, 109, 23, 750, 3, 142, 139, 5, 48, 64], [3, 19, 69, 730, 273, 62, 107, 187, 197, 351, 52, 14, 27, 9, 731, 5, 610, 15, 15, 59, 551, 272, 17, 22, 305, 8191, 52, 190, 13, 43, 335, 1821, 33, 22, 157, 107, 38, 13, 10, 121, 928, 12, 283, 2, 3, 572, 26, 52, 159, 560, 1367, 14, 524, 633, 9, 8, 178, 897, 73, 97, 3, 320, 11, 4, 633, 715, 2, 320, 1

In [9]:
type(tokenized_corpus)

list

In [10]:
def generate_data(corpus, vocab_size, window_size=2, sentence_batch_size=15,  batch_size=250):
    np.random.shuffle(np.array(corpus))
    number_of_sentence_batches = (len(corpus) // sentence_batch_size) + 1
    for batch in range(number_of_sentence_batches):
        lower_end = batch*batch_size
        upper_end = (batch+1)*batch_size if batch+1 < number_of_sentence_batches else len(corpus)
        mini_batch_size = upper_end - lower_end
        maxlen = window_size*2
        X = []
        Y = []
        for review_id, words in enumerate(corpus[lower_end:upper_end]):
            L = len(words)
            for index, word in enumerate(words):
                contexts = []
                labels   = []            
                s = index - window_size
                e = index + window_size + 1

                contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
                labels.append(word)

                x = pad_sequences(contexts, maxlen=maxlen)
                y = np_utils.to_categorical(labels, vocab_size)
                X.append(x)
                Y.append(y)
        X = tf.constant(X)
        Y = tf.constant(Y)
        number_of_batches = len(X) // batch_size
        for real_batch in range(number_of_batches):
          lower_end = batch*batch_size
          upper_end = (batch+1)*batch_size
          batch_X = tf.squeeze(X[lower_end:upper_end])
          batch_Y = tf.squeeze(Y[lower_end:upper_end])
          yield (batch_X, batch_Y)


In [11]:
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=4))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_dim,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='adam')
cbow.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 50)             997500    
                                                                 
 lambda (Lambda)             (None, 50)                0         
                                                                 
 dense (Dense)               (None, 19950)             1017450   
                                                                 
Total params: 2,014,950
Trainable params: 2,014,950
Non-trainable params: 0
_________________________________________________________________


In [12]:
def fit_model():
    if not BATCH:
        X, Y = generate_data(corpus=tokenized_corpus, vocab_size=vocab_size, batch_size=len(X))
        print(f'Size of X is {X.shape} and Y is {Y.shape}')
        cbow.fit(X, Y, epochs = epochs)
    else:
        index = 1
        for x, y in generate_data(corpus=tokenized_corpus, vocab_size=vocab_size, batch_size=batch_size):
            print(f'Training on Iteration: {index}')
            index += 1
            history = cbow.train_on_batch(x, y, reset_metrics=False, return_dict=True)
            print(history)
            if index > epochs:
                break

In [13]:
fit_model()

Training on Iteration: 1
{'loss': 9.901032447814941}
Training on Iteration: 2
{'loss': 9.89981460571289}
Training on Iteration: 3
{'loss': 9.89858341217041}
Training on Iteration: 4
{'loss': 9.89734172821045}
Training on Iteration: 5
{'loss': 9.89609146118164}
Training on Iteration: 6
{'loss': 9.894830703735352}
Training on Iteration: 7
{'loss': 9.893559455871582}
Training on Iteration: 8
{'loss': 9.8922758102417}
Training on Iteration: 9
{'loss': 9.89097785949707}
Training on Iteration: 10
{'loss': 9.889666557312012}
Training on Iteration: 11
{'loss': 9.888338088989258}
Training on Iteration: 12
{'loss': 9.886992454528809}
Training on Iteration: 13
{'loss': 9.885626792907715}
Training on Iteration: 14
{'loss': 9.884241104125977}
Training on Iteration: 15
{'loss': 9.882833480834961}
Training on Iteration: 16
{'loss': 9.881401062011719}
Training on Iteration: 17
{'loss': 9.879944801330566}
Training on Iteration: 18
{'loss': 9.878460884094238}
Training on Iteration: 19
{'loss': 9.8769493

In [14]:
with open('./cbow_scratch_synonims.txt' ,'w') as f:
    f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
    vectors = cbow.get_weights()[0]
    for word, i in tokenizer.word_index.items():
        str_vec = ' '.join(map(str, list(vectors[i, :])))
        f.write('{} {}\n'.format(word, str_vec))

In [15]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./cbow_scratch_synonims.txt', binary=False)



In [16]:
w2v.most_similar(positive=['gasoline'])

[('maintains', 0.5357844233512878),
 ('96', 0.5284336805343628),
 ("there're", 0.5171193480491638),
 ('chewing', 0.5127679109573364),
 ('crisscrosses', 0.496882826089859),
 ('funsports', 0.48369425535202026),
 ('biancoverde', 0.48104915022850037),
 ('rewards', 0.478218138217926),
 ('surf', 0.47760260105133057),
 ('chairs', 0.47078537940979004)]

In [17]:
w2v.most_similar(negative=['apple'])

[('rowdy', 0.519658088684082),
 ('glom', 0.5132775902748108),
 ('bedroom', 0.49135029315948486),
 ('refresher', 0.4903404712677002),
 ('shirt', 0.4868806004524231),
 ('dali', 0.4821418821811676),
 ('yankin', 0.4679061472415924),
 ('clip', 0.46546414494514465),
 ('flakey', 0.4637853801250458),
 ('grandpa', 0.4583456218242645)]